Вариант, который я собрал изначально. Не подумал даже про global

In [18]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification, make_regression
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [55]:
import time
# Построение дерева с помощью рекурсивной функции

def build_tree(data, labels, leaf_m = 3, leaf_n = 0):

    gain, t, index = find_best_split(data, labels)
    
    leaf_i = leaf_n

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if gain == 0:
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
    
    leaf_i += 1
    
    if leaf_i <= leaf_m:
        continue
        
    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels, leaf_m, leaf_i)

#     print(time.time(), true_branch)
    false_branch = build_tree(false_data, false_labels, leaf_m, leaf_i)
    
#     print(time.time(), false_branch)
    
    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch, leaf_m, leaf_i)

SyntaxError: 'continue' not properly in loop (<ipython-input-55-5bbe7249606a>, line 19)

Скопировал базовую часть

In [1]:
# Класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [3]:
# Класс листа

class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
        # подсчет количества объектов разных классов
        classes = {}  # сформируем словарь "класс: количество объектов"
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
        #  найдем класс, количество объектов которого будет максимальным в этом листе и вернем его    
        prediction = max(classes, key=classes.get)
        return prediction        

In [4]:
# Расчет критерия Джини

def gini(labels):
    #  подсчет количества объектов разных классов
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    #  расчет критерия
    impurity = 1
    for label in classes:
        p = classes[label] / len(labels)
        impurity -= p ** 2
        
    return impurity

In [86]:
# Расчет прироста

def gain(left_labels, right_labels, root_gini):

    # доля выборки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return root_gini - p * gini(left_labels) - (1 - p) * gini(right_labels)

In [5]:
# Расчет качества

def quality(left_labels, right_labels, current_gini):

    # доля выбоки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return current_gini - p * gini(left_labels) - (1 - p) * gini(right_labels)

In [6]:
# Разбиение датасета в узле

def split(data, labels, index, t):
    
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)
        
    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

In [7]:
# Нахождение наилучшего разбиения

def find_best_split(data, labels):
    
    #  обозначим минимальное количество объектов в узле
    min_leaf = 5
    
    current_gini = gini(labels)

    best_quality = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    for index in range(n_features):
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique([row[index] for row in data])
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue
            
            current_quality = quality(true_labels, false_labels, current_gini)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index

In [8]:
# Построение дерева с помощью рекурсивной функции, базовый случай

def build_tree(data, labels):

    quality, t, index = find_best_split(data, labels)

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if quality == 0:
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels)
    false_branch = build_tree(false_data, false_labels)

    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [9]:
def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

In [10]:
def predict(data, tree):
    
    classes = []
    for obj in data:
        prediction = classify_object(obj, tree)
        classes.append(prediction)
    return classes

In [14]:
# Напечатаем ход нашего дерева
def print_tree(node, spacing=""):

    # Если лист, то выводим его прогноз
    if isinstance(node, Leaf):
        print(spacing + "Прогноз:", node.prediction)
        return

    # Выведем значение индекса и порога на этом узле
    print(spacing + 'Индекс', str(node.index))
    print(spacing + 'Порог', str(node.t))

    # Рекурсионный вызов функции на положительном поддереве
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Рекурсионный вызов функции на положительном поддереве
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [33]:
# Введем функцию подсчета точности как доли правильных ответов
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

def check_accuracy(my_tree):
    train_answers = predict(train_data, my_tree)

    train_accuracy = accuracy_metric(train_labels, train_answers)
    print(f'Доля правильных ответов на обучающей выборке: {train_accuracy:.2f}')

    answers = predict(test_data, my_tree)

    test_accuracy = accuracy_metric(test_labels, answers)
    print(f'Доля правильных ответов на обучающей выборке: {test_accuracy:.2f}')

In [19]:
# сгенерируем данные
classification_data, classification_labels = make_classification(n_samples=1000, n_features=2, n_informative=2, 
                                                                 n_classes=2, n_redundant=0,
                                                                 n_clusters_per_class=1, random_state=5)

In [21]:
# Разобьем выборку на обучающую и тестовую

from sklearn.model_selection import train_test_split

train_data, test_data, train_labels, test_labels = train_test_split(classification_data, 
                                                                    classification_labels, 
                                                                    test_size=0.3,
                                                                    random_state=1)

In [22]:
my_tree = build_tree(train_data, train_labels)
print_tree(my_tree)

Индекс 0
Порог -0.001967524769128759
--> True:
  Индекс 1
  Порог -1.3993975578815423
  --> True:
    Индекс 1
    Порог -1.6646404646216442
    --> True:
      Индекс 0
      Порог -0.8728986138474495
      --> True:
        Прогноз: 0
      --> False:
        Прогноз: 0
    --> False:
      Индекс 0
      Порог -0.6882680999463433
      --> True:
        Прогноз: 0
      --> False:
        Индекс 0
        Порог -0.40118216125291906
        --> True:
          Индекс 1
          Порог -1.568473711740985
          --> True:
            Индекс 0
            Порог -0.5543283463967665
            --> True:
              Прогноз: 1
            --> False:
              Прогноз: 1
          --> False:
            Прогноз: 0
        --> False:
          Индекс 1
          Порог -1.5264301097909836
          --> True:
            Прогноз: 1
          --> False:
            Индекс 1
            Порог -1.4255078037225668
            --> True:
              Прогноз: 1
            --> False:
    

In [34]:
check_accuracy(my_tree)

Доля правильных ответов на обучающей выборке: 98.14
Доля правильных ответов на обучающей выборке: 95.00


In [58]:
def build_tree_1(data, labels):
    
    global max_leaves, n_questions, num_leaves
    
    gain, t, index = find_best_split(data, labels)

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    # или когда количество листьев стало больше максимального
    # print(num_leaves, n_questions)
    if gain == 0 or num_leaves >= max_leaves - 1 or n_questions >= max_leaves - 1:
        num_leaves += 1
        return Leaf(data, labels)

    n_questions += 1
    
    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels)
    false_branch = build_tree(false_data, false_labels)

    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [59]:
num_leaves = 0
n_questions = 0
max_leaves = 4

my_tree = build_tree_1(train_data, train_labels)
print_tree(my_tree)

Индекс 0
Порог -0.001967524769128759
--> True:
  Индекс 1
  Порог -1.3993975578815423
  --> True:
    Индекс 1
    Порог -1.6646404646216442
    --> True:
      Прогноз: 0
    --> False:
      Прогноз: 1
  --> False:
    Прогноз: 0
--> False:
  Прогноз: 1


In [57]:
check_accuracy(my_tree)

Доля правильных ответов на обучающей выборке: 89.86
Доля правильных ответов на обучающей выборке: 86.00


In [71]:
# Построение дерева с помощью рекурсивной функции
def build_tree_2(data, labels, max_depth):
    
    gain, t, index = find_best_split(data, labels)

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    #  или когда достигли максимальной глубины дерева
    if gain == 0 or max_depth == 0:
        return Leaf(data, labels)
    
    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
    
    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels, max_depth - 1)
    false_branch = build_tree(false_data, false_labels, max_depth - 1)

    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [74]:
my_tree = build_tree_2(train_data, train_labels, max_depth=3)
print_tree(my_tree)

Индекс 0
Порог -0.001967524769128759
--> True:
  Индекс 1
  Порог -1.3993975578815423
  --> True:
    Индекс 1
    Порог -1.6646404646216442
    --> True:
      Прогноз: 0
    --> False:
      Прогноз: 1
  --> False:
    Прогноз: 0
--> False:
  Индекс 1
  Порог -1.4518330557811816
  --> True:
    Прогноз: 0
  --> False:
    Индекс 0
    Порог 0.08948763365897316
    --> True:
      Прогноз: 1
    --> False:
      Прогноз: 1


**2. *Реализуйте дерево для задачи регрессии. Возьмите за основу дерево, реализованное в методичке, заменив механизм предсказания в листе на взятие среднего значения по выборке, и критерий Джини на дисперсию значений.**

In [81]:
data, targets = make_regression(n_features=2, n_informative=2, random_state=5)
from sklearn.model_selection import train_test_split
train_data_regr, test_data_regr, train_target_regr, test_target_regr = train_test_split(data, 
                                                                                        targets, 
                                                                                        test_size=0.3,
                                                                                        random_state=1)

In [82]:
class Leaf:
    
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets
        self.prediction = self.predict()
        
    def predict(self):
        return self.targets.mean()

In [77]:
def mse(targets):
    return np.mean((targets - targets.mean())**2)

In [83]:
def find_best_split(data, targets):
    
    #  обозначим минимальное количество объектов в узле
    min_samples_leaf = 5

    root_mse = mse(targets)

    best_gain = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    for index in range(n_features):
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique(data[:, index])
        
        for t in t_values:
            true_data, false_data, true_targets, false_targets = split(data, targets, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_samples_leaf or len(false_data) < min_samples_leaf:
                continue
            
            current_gain = gain(true_targets, false_targets, root_mse)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_gain > best_gain:
                best_gain, best_t, best_index = current_gain, t, index

    return best_gain, best_t, best_index

In [84]:
def build_tree(data, target):

    gain, t, index = find_best_split(data, target)

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if gain == 0:
        return Leaf(data, target)

    true_data, false_data, true_target, false_target = split(data, target, index, t)

    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_target)

    false_branch = build_tree(false_data, false_target)

    node = Node(index, t, true_branch, false_branch)

    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return node

In [87]:
my_tree = build_tree(train_data_regr, train_target_regr)
print_tree(my_tree)

Индекс 0
Порог -1.167278449710173
--> True:
  Прогноз: -121.39064849773837
--> False:
  Индекс 0
  Порог -0.8126247611063044
  --> True:
    Прогноз: -58.83807643952095
  --> False:
    Индекс 0
    Порог -0.5017555471945
    --> True:
      Прогноз: -50.68338930141631
    --> False:
      Индекс 0
      Порог -0.28143012121166766
      --> True:
        Прогноз: -19.53457742091028
      --> False:
        Индекс 0
        Порог 0.10134479014204936
        --> True:
          Индекс 0
          Порог -0.10593044205742323
          --> True:
            Прогноз: -19.179384142931184
          --> False:
            Прогноз: 11.007437547567205
        --> False:
          Индекс 0
          Порог 0.30364846530823975
          --> True:
            Прогноз: 17.82546436535075
          --> False:
            Индекс 0
            Порог 0.6178447508392777
            --> True:
              Индекс 0
              Порог 0.44295626086393586
              --> True:
                Прогноз: 32.35

In [88]:
from sklearn.metrics import r2_score

train_answers = predict(train_data_regr, my_tree)
train_r2 = r2_score(train_target_regr, train_answers)
print(train_r2)

answers = predict(test_data_regr, my_tree)
test_r2 = r2_score(test_target_regr, answers)
print(test_r2)

0.8994102006971118
0.8047526472601623
